In [25]:
import boto3
import json
from PIL import Image
import io
import os
import random
import matplotlib.pyplot as plt
import time
from botocore.config import Config
from pathlib import Path

# Create clients with longer timeout
config = Config(
    read_timeout=20,
    connect_timeout=20,
    retries={'max_attempts': 2}
)
runtime = boto3.client('sagemaker-runtime', config=config)
sagemaker_client = boto3.client('sagemaker')

def get_random_test_image():
    """
    Get a random image from the sample data
    Returns:
        tuple: (image_path, true_label)
    """
    # Debug: Print current working directory
    print(f"Current working directory: {os.getcwd()}")
    
    # Convert relative path to absolute path from notebook location
    notebook_dir = Path(os.getcwd())
    print(f"Notebook directory: {notebook_dir}")
    
    project_root = notebook_dir.parent
    print(f"Project root: {project_root}")
    
    # Point to sample_data directory
    data_path = project_root / 'data/sample_data'
    print(f"Looking for data in: {data_path}")
    print(f"Data path exists? {data_path.exists()}")
    
    # Debug: Show available files
    if data_path.exists():
        print(f"Contents of sample_data:")
        print(os.listdir(data_path))
    
    # Get all image files
    # images = [f for f in os.listdir(data_path) if f.endswith(('.jpg', '.jpeg', '.png'))]
    # print(f"Found images: {images}")
    
    # Choose a random image
    # image_name = random.choice(images)
    image_path = data_path / "dog.jpg"
    
    # Determine label from filename
    # true_label = 'cat' if 'cat' in image_name.lower() else 'dog'
    true_label = "dog"
    
    print(f"Selected image: {image_path}")
    print(f"Image exists? {image_path.exists()}")
    print(f"Label: {true_label}")
    
    return str(image_path), true_label


def test_prediction_verbose(image_path):
    """Test prediction with verbose logging"""
    try:
        # First check endpoint status
        if not check_endpoint_status('cat-dog-classifier-v2'):
            print("Endpoint is not ready!")
            return
            
        # Load and resize image
        print("Loading and preprocessing image...")
        img = Image.open(image_path)
        
        # Resize image to reduce size
        img = img.resize((224, 224))
        
        # Convert to bytes with reduced quality
        img_byte_arr = io.BytesIO()
        img.save(img_byte_arr, format='JPEG', quality=85)
        image_data = img_byte_arr.getvalue()
        print(f"Processed image size: {len(image_data)} bytes")
        
        print("\nMaking prediction request...")
        start_time = time.time()
        
        # Create new runtime client with longer timeout
        runtime_config = Config(
            read_timeout=30,  # Increased timeout
            connect_timeout=30,
            retries={'max_attempts': 3}
        )
        runtime = boto3.client('sagemaker-runtime', config=runtime_config)
        
        # Make prediction
        try:
            response = runtime.invoke_endpoint(
                EndpointName='cat-dog-classifier-v2',
                ContentType='application/x-image',
                Body=image_data
            )
            
            print(f"Request time: {time.time() - start_time:.2f} seconds")
            print("Response received!")
            
            # Parse response
            result = json.loads(response['Body'].read().decode())
            print("\nPrediction Results:")
            print("-----------------")
            print(f"Cat Probability: {result['cat_probability']:.2%}")
            print(f"Dog Probability: {result['dog_probability']:.2%}")
            
            return result
            
        except runtime.exceptions.ModelError as e:
            print(f"Model error: {e}")
        except Exception as e:
            print(f"\nError during endpoint invocation:")
            print(f"Type: {type(e).__name__}")
            print(f"Message: {str(e)}")
            
    except Exception as e:
        print(f"\nError during image processing:")
        print(f"Type: {type(e).__name__}")
        print(f"Message: {str(e)}")
    
    return None


def check_endpoint_detailed():
    """Detailed endpoint status check"""
    client = boto3.client('sagemaker')
    
    try:
        # Get endpoint details
        endpoint = client.describe_endpoint(EndpointName='cat-dog-classifier-v2')
        
        print("\nEndpoint Details:")
        print("-----------------")
        print(f"Status: {endpoint['EndpointStatus']}")
        print(f"Created: {endpoint['CreationTime']}")
        print(f"Last Modified: {endpoint['LastModifiedTime']}")
        print(f"Failure Reason: {endpoint.get('FailureReason', 'None')}")
        
        # Get endpoint config details
        config = client.describe_endpoint_config(EndpointConfigName=endpoint['EndpointConfigName'])
        
        print("\nEndpoint Configuration:")
        print("----------------------")
        print(f"Instance Type: {config['ProductionVariants'][0]['InstanceType']}")
        print(f"Instance Count: {config['ProductionVariants'][0]['InitialInstanceCount']}")
        
        # Get CloudWatch logs URL
        region = boto3.session.Session().region_name
        print(f"\nCloudWatch Logs URL:")
        print(f"https://{region}.console.aws.amazon.com/cloudwatch/home?region={region}#logStream:group=/aws/sagemaker/Endpoints/cat-dog-classifier-v2")
        
        return endpoint['EndpointStatus'] == 'InService'
        
    except Exception as e:
        print(f"Error checking endpoint: {e}")
        return False


In [26]:
# Test the endpoint
print("Starting endpoint test...\n")

is_ready = check_endpoint_detailed()

# Get a random test image
image_path, true_label = get_random_test_image()
print(f"Testing with a {true_label} image: {image_path}")

# Run prediction
result = test_prediction_verbose(image_path)

Starting endpoint test...


Endpoint Details:
-----------------
Status: InService
Created: 2025-03-06 20:08:36.415000+00:00
Last Modified: 2025-03-06 20:13:23.571000+00:00
Failure Reason: None

Endpoint Configuration:
----------------------
Instance Type: ml.t2.large
Instance Count: 1

CloudWatch Logs URL:
https://us-east-1.console.aws.amazon.com/cloudwatch/home?region=us-east-1#logStream:group=/aws/sagemaker/Endpoints/cat-dog-classifier-v2
Current working directory: /home/sagemaker-user/src/notebooks
Notebook directory: /home/sagemaker-user/src/notebooks
Project root: /home/sagemaker-user/src
Looking for data in: /home/sagemaker-user/src/data/sample_data
Data path exists? True
Contents of sample_data:
['dog.jpg']
Selected image: /home/sagemaker-user/src/data/sample_data/dog.jpg
Image exists? True
Label: dog
Testing with a dog image: /home/sagemaker-user/src/data/sample_data/dog.jpg
Endpoint status: InService
Last modified: 2025-03-06 20:13:23.571000+00:00
Instance type: cat-dog-classi